# Imports

In [1]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

from keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, BatchNormalization, Rescaling, Dropout, Resizing
from keras.layers import RandomFlip, RandomTranslation, RandomRotation, RandomZoom
from keras.models import Sequential
from keras.metrics import F1Score, Precision
from keras.losses import CategoricalFocalCrossentropy, BinaryFocalCrossentropy
from keras.callbacks import EarlyStopping
from keras.utils import to_categorical

from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE, ADASYN, KMeansSMOTE
from imblearn.under_sampling import TomekLinks, RepeatedEditedNearestNeighbours
from imblearn.combine import SMOTETomek, SMOTEENN
from sklearn.utils import compute_class_weight
from keras.utils import to_categorical
import os
from PIL import Image

2025-04-16 21:26:35.298630: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-04-16 21:26:35.455090: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-04-16 21:26:35.585213: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1744831595.710393   10014 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1744831595.745120   10014 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1744831596.011542   10014 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linkin

# Move images into class folders

In [ ]:
def move_and_separate_images(list_of_folders, df_metadata):
    df_metadata = df_metadata.set_index('image_id', drop=True)
    count = 0
    for folder in list_of_folders:
        for root, dirs, files in os.walk(folder):
            for file in files:
                try:
                    os.makedirs(f"dataset/{df_metadata.loc[file.strip('.jpg'), 'dx']}")
                except:
                    source=os.path.join(root, file)
                    destination=os.path.join('dataset', df_metadata.loc[file.strip('.jpg'), 'dx'], file)
                    os.rename(source,destination)
                    count += 1
                    if count%100 == 0:
                        print(f'{count} images were processed')

In [ ]:
df = pd.read_csv('HAM10000_metadata.csv')

move_and_separate_images(['HAM10000_images_part_1', 'HAM10000_images_part_2'], df)

# Move images into cancerous/non cancerous folders

In [2]:
class_mapping = {
    "akiec": "canc",
    "bcc": "canc",
    "bkl": "nocanc",
    "df": "nocanc",
    "mel": "canc",
    "nv": "nocanc",
    "vasc": "nocanc"
}

In [3]:
def move_and_separate_images(list_of_folders, df_metadata):
    df_metadata = df_metadata.set_index('image_id', drop=True)
    count = 0
    for folder in list_of_folders:
        for root, dirs, files in os.walk(folder):
            for file in files:
                try:
                    os.makedirs(f"dataset/{class_mapping[df_metadata.loc[file.strip('.jpg'), 'dx']]}")
                except:
                    source=os.path.join(root, file)
                    destination=os.path.join('dataset', class_mapping[df_metadata.loc[file.strip('.jpg'), 'dx']], file)
                    os.rename(source,destination)
                    count += 1
                    if count%100 == 0:
                        print(f'{count} images were processed')

In [4]:
df = pd.read_csv('HAM10000_metadata.csv')

move_and_separate_images(['HAM10000_images_part_1', 'HAM10000_images_part_2'], df)

100 images were processed
200 images were processed
300 images were processed
400 images were processed
500 images were processed
600 images were processed
700 images were processed
800 images were processed
900 images were processed
1000 images were processed
1100 images were processed
1200 images were processed
1300 images were processed
1400 images were processed
1500 images were processed
1600 images were processed
1700 images were processed
1800 images were processed
1900 images were processed
2000 images were processed
2100 images were processed
2200 images were processed
2300 images were processed
2400 images were processed
2500 images were processed
2600 images were processed
2700 images were processed
2800 images were processed
2900 images were processed
3000 images were processed
3100 images were processed
3200 images were processed
3300 images were processed
3400 images were processed
3500 images were processed
3600 images were processed
3700 images were processed
3800 image

# Make arrays

In [6]:
def array_from_images(folder, df_metadata, dict_of_labels, h=224, w=224, channels=3):
    # Create an array of images and labels the size of the number of pictures
    nb_files = 0
    for root, dirs, files in os.walk(folder):
        for file in files:
            nb_files += 1
    array = np.zeros(shape=(nb_files, h, w, channels))
    labels = np.zeros(shape=(nb_files,))

    # Check the name and fill array and labels
    df_metadata = df_metadata.set_index('image_id', drop=True)
    count = 0

    for root, dirs, files in os.walk(folder):
        for file in files:
            with Image.open(os.path.join(root, file)) as im:
                array[count,:,:,:] = np.asarray(im.resize((h,w)))
                try:
                    labels[count] = dict_of_labels[class_mapping[df_metadata.loc[file.strip('.jpg'), 'dx']]]
                except:
                    labels[count] = dict_of_labels[df_metadata.loc[file.strip('.jpg'), 'dx']]
                count += 1
                if count%100 == 0:
                    print(f'{count} images were processed')
    return array, labels

In [7]:
df = pd.read_csv('HAM10000_metadata.csv')
dict_label = {k:v for k,v in zip(['nocanc', 'canc'], range(7))}

size = 64

X, y = array_from_images('dataset/', df, dict_label, h=size, w=size)


100 images were processed
200 images were processed
300 images were processed
400 images were processed
500 images were processed
600 images were processed
700 images were processed
800 images were processed
900 images were processed
1000 images were processed
1100 images were processed
1200 images were processed
1300 images were processed
1400 images were processed
1500 images were processed
1600 images were processed
1700 images were processed
1800 images were processed
1900 images were processed
2000 images were processed
2100 images were processed
2200 images were processed
2300 images were processed
2400 images were processed
2500 images were processed
2600 images were processed
2700 images were processed
2800 images were processed
2900 images were processed
3000 images were processed
3100 images were processed
3200 images were processed
3300 images were processed
3400 images were processed
3500 images were processed
3600 images were processed
3700 images were processed
3800 image

In [10]:
np.savez_compressed(f'/home/dimitri/Projects/SK-MNIST/X-{size}x{size}', X, allow_pickle=True)
np.savez_compressed(f'/home/dimitri/Projects/SK-MNIST/y-{size}x{size}', y, allow_pickle=True)

# Load arrays

In [ ]:
X = np.load('X-64x64.npz')['arr_0']
y = np.load('y-64x64.npz')['arr_0']

# Split arrays

In [11]:
from sklearn.model_selection import train_test_split

X_train, X_temp, y_train, y_temp = train_test_split(X, y, stratify=y, test_size=0.2, random_state=38)



X_test, X_val, y_test, y_val = train_test_split(X_temp, y_temp, stratify=y_temp, test_size=0.5, random_state=38)

del X_temp, y_temp

# SMOTETOMEK

In [ ]:
shape_origin = X_train.shape

X_train = np.reshape(X_train, (shape_origin[0], shape_origin[1]*shape_origin[2]*shape_origin[3]))

smotetomek = SMOTETomek(random_state=38, n_jobs=-1)

X_train, y_train = smotetomek.fit_resample(X_train, y_train)

X_train = np.reshape(X_train, (X_train.shape[0], shape_origin[1], shape_origin[2], shape_origin[3]))



# One Hot Encoding of labels

In [12]:
y_train = to_categorical(y_train)
y_val = to_categorical(y_val)
y_test = to_categorical(y_test)

# Helper functions

In [13]:
def get_earlystopping(patience=10):
    early_stopping = EarlyStopping(
    monitor='val_f1_score',
    patience=patience,
    verbose=1,
    restore_best_weights=True)
    return early_stopping

In [14]:
def plot_learning_curves(model):
    fig, ax = plt.subplots(1,3, figsize=(15,5))
    ax[0].plot(model.history.history['val_f1_score'], label='val_f1_score')
    ax[0].plot(model.history.history['f1_score'], label='f1_score')
    ax[0].legend()
    try:
        ax[1].plot(model.history.history['val_accuracy'], label='val_accuracy')
        ax[1].plot(model.history.history['accuracy'], label='accuracy')
        ax[1].legend()
    except:
        ax[1].plot(model.history.history['val_precision_1'], label='val_precision_1')
        ax[1].plot(model.history.history['precision_1'], label='precision_1')
        ax[1].legend()
    ax[2].plot(model.history.history['val_loss'], label='val_loss')
    ax[2].plot(model.history.history['loss'], label='loss')
    ax[2].legend()
    fig.show;

In [15]:
def get_analysis(model, testX, testy):
    plot_learning_curves(model)
    loss, acc, f1 = model.evaluate(testX, testy)
    print(f'The model gave')
    print(f'Loss: {loss:.2f}')
    print(f'Accuracy: {acc:.2f}')
    print(f'F1 Macro: {f1:.2f}')
    y_pred = model.predict(testX)
    y_res = np.argmax(y_pred, axis=1)
    print(classification_report(testy, y_res))
    return y_pred, y_res

In [16]:
def get_analysis_cat(model, testX, testy):
    plot_learning_curves(model)
    loss, acc, f1 = model.evaluate(testX, testy)
    print(f'The model gave')
    print(f'Loss: {loss:.2f}')
    print(f'Accuracy: {acc:.2f}')
    print(f'F1 Macro: {f1:.2f}')
    predy = model.predict(testX)
    resy = to_categorical(np.argmax(predy, axis=1))
    print(classification_report(testy,resy))
    return predy, resy

In [17]:
def compile_and_train(model, loss, opt, metrics, epochs, patience=None, steps=None):
    model.compile(loss=loss,
                optimizer=opt,
                metrics=metrics)

    model.summary()

    if patience != None:
        model.fit(
            X_train,
            y_train,
            epochs=epochs,
            validation_data=(X_val, y_val),
            callbacks=[get_earlystopping(patience)],
            steps_per_epoch=steps
            )
    else:
        model.fit(
            X_train,
            y_train,
            epochs=epochs,
            validation_data=(X_val, y_val),
            steps_per_epoch=steps
            )

    return model

# Model

In [18]:
filters = 16

In [19]:
model_020 = Sequential([
    Input(shape=(X_train.shape[1:])),
    Rescaling(1./255),
    RandomFlip('horizontal and vertical'),
    RandomRotation(factor=(-0.3, 0.3)),
    RandomTranslation(height_factor=(-0.3,0.3), width_factor=(-0.3, 0.3)),
    RandomZoom(height_factor=(-0.3,0.3), width_factor=(-0.3, 0.3)),
    Conv2D(filters, kernel_size=3, padding='SAME', activation='relu'),
    Conv2D(filters, kernel_size=3, padding='SAME', activation='relu'),
    Conv2D(filters, kernel_size=3, padding='SAME', activation='relu'),
    MaxPooling2D(pool_size=2),
    BatchNormalization(),
    Dropout(0.2),
    Conv2D(filters*2, kernel_size=3, padding='SAME', activation='relu'),
    Conv2D(filters*2, kernel_size=3, padding='SAME', activation='relu'),
    Conv2D(filters*2, kernel_size=3, padding='SAME', activation='relu'),
    MaxPooling2D(pool_size=2),
    BatchNormalization(),
    Dropout(0.2),
    Conv2D(filters*4, kernel_size=3, padding='SAME', activation='relu'),
    Conv2D(filters*4, kernel_size=3, padding='SAME', activation='relu'),
    Conv2D(filters*4, kernel_size=3, padding='SAME', activation='relu'),
    MaxPooling2D(pool_size=2),
    BatchNormalization(),
    Dropout(0.2),
    Conv2D(filters*8, kernel_size=3, padding='SAME', activation='relu'),
    Conv2D(filters*8, kernel_size=3, padding='SAME', activation='relu'),
    Conv2D(filters*8, kernel_size=3, padding='SAME', activation='relu'),
    MaxPooling2D(pool_size=2),
    BatchNormalization(),
    Dropout(0.2),
    Flatten(),
    Dense(filters, activation='relu'),
    Dense(2, activation='softmax')
])

history020 = compile_and_train(model_020,
                               loss=BinaryFocalCrossentropy(apply_class_balancing=True),
                               opt='adam',
                               metrics=[F1Score(average='macro')],
                               epochs=200,
                               patience=10)

2025-04-16 21:34:35.264373: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ rescaling (Rescaling)           │ (None, 64, 64, 3)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ random_flip (RandomFlip)        │ (None, 64, 64, 3)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ random_rotation                 │ (None, 64, 64, 3)      │             0 │
│ (RandomRotation)                │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ random_translation              │ (None, 64, 64, 3)      │             0 │
│ (RandomTranslation)             │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ random_zoom (RandomZoom)        │ (None, 64, 64, 3)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d (Conv2D)                 │ (None, 64, 64, 16)     │           448 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 64, 64, 16)     │         2,320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 64, 64, 16)     │         2,320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 32, 32, 16)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 32, 32, 16)     │            64 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 32, 32, 16)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 32, 32, 32)     │         4,640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 32, 32, 32)     │         9,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 32, 32, 32)     │         9,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 16, 16, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 16, 16, 32)     │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 16, 16, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_6 (Conv2D)               │ (None, 16, 16, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_7 (Conv2D)               │ (None, 16, 16, 64)     │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_8 (Conv2D)               │ (None, 16, 16, 64)     │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 8, 8, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 8, 8, 64)       │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼─────────────

 Total params: 523,378 (2.00 MB)

 Trainable params: 522,898 (1.99 MB)

 Non-trainable params: 480 (1.88 KB)

Epoch 1/200
 63/251 ━━━━━━━━━━━━━━━━━━━━ 2:20 748ms/step - f1_score: 0.5012 - loss: 0.0728

KeyboardInterrupt: 

In [ ]:
get_analysis_cat(model_020, X_test, y_test)

## Custom loss

In [ ]:
import tensorflow as tf
from keras.losses import CategoricalFocalCrossentropy
from keras import backend as K
from keras import ops

def f1macro(y_true, y_pred):
    tp = []
    fp = []
    fn = []
    precision = []
    recall = []
    f1 = []

    cfc = CategoricalFocalCrossentropy()


    for n in range(7):
        tp.append(ops.sum(ops.cast(y_true[:, n] * y_pred[:, n], 'float32'), axis=0))
        fp.append(ops.sum(ops.cast((1 - y_true[:, n]) * y_pred[:, n], 'float32'), axis=0))
        fn.append(ops.sum(ops.cast(y_true[:, n] * (1 - y_pred[:, n]), 'float32'), axis=0))

    for n in range(7):
        precision.append(tp[n] / (tp[n] + fp[n] + K.epsilon()))
        recall.append(tp[n] / (tp[n] + fn[n] + K.epsilon()))

    for n in range(7):
        f1.append(2 * (precision[n] * recall[n]) / (precision[n] + recall[n] + K.epsilon()))

    f1macro = tf.stack(f1)
    f1macroscore = ops.mean(f1macro)

    f1macroscore = tf.where(tf.math.is_nan(f1macroscore), tf.zeros_like(f1macroscore), f1macroscore)

    return cfc + (1 - f1macroscore)